# Retrieval-Augmented Generation (RAG) with open-source Hugging Face LLMs using LangChain

![RAG pic](images/RAG.png)

## Introduction: 

**Retrieval-Augmented Generation (RAG)** is an approach in natural language processing (NLP) that enhances the capabilities of generative models by integrating external knowledge retrieval into the generation process. This technique aims to improve the quality, relevance, and factual accuracy of the generated text by allowing the model to dynamically access and incorporate information from a large corpus of documents or databases during the generation task. The process involves two key components: a retrieval system and a generative model.

**Working Mechanism**

The working mechanism of RAG typically involves the following steps:

- Query Formation: The system formulates a query based on the initial input or prompt. This query is designed to retrieve information that is likely to be relevant to generating the desired output.

- Information Retrieval: The formulated query is used to fetch relevant information from an external database or knowledge base. The retrieval system may return one or more documents, passages, or data entries that match the query.

- Content Integration: The retrieved information, along with the original input, is provided to the generative model. The model then integrates this information to produce a coherent and contextually enriched output.

- Generation: The generative model synthesizes the final text, taking into account both the input and the retrieved external information. This step ensures that the output is not only relevant and informative but also maintains a natural and fluent language style.

Let's get started!

## Library installation
- Create a virtual environment and install the necessary python libraries
- `pip install transformers sentence-tranformers langchain torch faiss-cpu numpy`

## Library configuration

In [1]:
pip install numpy langchain_community langchain

You should consider upgrading via the 'c:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

## Document preparation
**We are going to download 4 publications from United States Census Bureau on the following topics:**
- Occupation, Earnings, and Job Characteristics: July 2022
- Household Income in States andMetropolitan Areas: 2022
- Poverty in States and Metropolitan Areas: 2022
- Health Insurance Coverage Status and Type by Geography: 2021 and 2022

We prepare this documents for the LLM to use as a knowledge base.

In [3]:
# Download documents from U.S. Census Bureau to local directory.
os.makedirs("us_census", exist_ok=True)
files = [
    "https://www.census.gov/content/dam/Census/library/publications/2022/demo/p70-178.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-017.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-016.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-015.pdf",
]
for url in files:
    file_path = os.path.join("us_census", url.rpartition("/")[2])
    urlretrieve(url, file_path)

**Split documents to smaller chunks** 

Documents should be: 
- large enough to contain enough information to answer a question, and 
- small enough to fit into the LLM prompt: Mistral-7B-v0.1 input tokens limited to 4096 tokens
- small enough to fit into the embeddings model: BAAI/bge-small-en-v1.5: input tokens limited to 512 tokens (roughly 2000 characters. Note: 1 token ~ 4 characters).

For this project, we are going to split documents to chunks of roughly 700 characters with an overlap of 50 characters.

In [4]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
# Load pdf files in the local directory
loader = PyPDFDirectoryLoader("./us_census/")

docs_before_split = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap  = 50,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

In [6]:
docs_after_split[0]

Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium')

In [7]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)
avg_char_before_split = avg_doc_length(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)

print(f'Before split, there were {len(docs_before_split)} documents loaded, with average characters equal to {avg_char_before_split}.')
print(f'After split, there were {len(docs_after_split)} documents (chunks), with average characters equal to {avg_char_after_split} (average chunk length).')

Before split, there were 63 documents loaded, with average characters equal to 3840.
After split, there were 398 documents (chunks), with average characters equal to 624 (average chunk length).


## Text Embeddings with Hugging Face Embedding Models
At the time of writing, there are 213 text embeddings models for English on the [Massive Text Embedding Benchmark (MTEB) leaderboard](https://huggingface.co/spaces/mteb/leaderboard). For our project, we are using LangChain's HuggingFaceBgeEmbeddings (BGE models on the Hugging Face), which according to LangChain are "the best open-source embedding models". Currently, **BAAI/bge-small-en-v1.5** model is the 26th on MTEB leaderboard with max tokens: 512 tokens, embedding dimensions: 384 and model size: 0.13GB.

In [8]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
    model_kwargs={'device':'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_25992\2508658439.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceBgeEmbeddings(
c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we can see how a sample embedding would look like for one of those chunks.

In [10]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [-7.50839636e-02 -1.18847443e-02 -3.14887911e-02  2.94039026e-02
  5.03487028e-02  5.62426485e-02 -1.69077870e-02  3.46888378e-02
 -9.79062095e-02 -2.52804272e-02  7.62901381e-02  5.73797785e-02
 -2.42905673e-02 -3.06788832e-02  6.20477134e-03  4.02186625e-02
 -8.71844776e-03 -8.25363584e-03 -3.58134210e-02  3.61376293e-02
 -4.81582806e-02  4.19040658e-02 -3.68844345e-02 -5.38902543e-02
  1.65312067e-02  1.20014157e-02 -1.46691557e-02  2.12824102e-02
 -5.34791276e-02 -1.49481431e-01  2.32283957e-03  3.20233852e-02
 -5.21530434e-02 -2.28870604e-02  2.32763253e-02  2.14958824e-02
 -1.39637133e-02  7.53695145e-02  5.07269688e-02  5.50112799e-02
 -3.30469683e-02  1.79151092e-02 -2.08278261e-02  1.26923830e-03
 -2.61849277e-02  2.41151522e-03 -1.92517135e-02  3.06514162e-03
  1.50287768e-03 -5.14435731e-02  3.78552377e-02 -1.38906231e-02
  4.22979072e-02  6.64815381e-02  6.62789345e-02 -4.27095369e-02
  9.69374366e-03 -3.63818146e-02 -4.47576195e-02  3

## Retrieval System for vector embeddings
Once we have a embedding model, we are ready to vectorize all our documents and store them in a vector store to construct a retrieval system. With specifically designed searching algorithms, a retrieval system can do similarity searching efficiently to retrieve relevant documents.

FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other. It solves limitations of traditional query search engines that are optimized for hash-based searches, and provides more scalable similarity search functions (nearest-neighbor search implementations).

In [11]:
pip install faiss-cpu

You should consider upgrading via the 'c:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [12]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

In [13]:
query = """What were the trends in median household income across different states in the United States between 2021 and 2022."""  # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query)
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 4 documents retrieved which are relevant to the query. Display the first one:

Comparisons
The U.S. median household income 
in 2022 was $74,755, according 
Figure 1.
Median Household Income in the Past 12 Months in the United States: 2005–2022
 
Note: Estimates for 2020 experimental data not shown. For more information on the 2020 experimental data products, 
refer to <www.census.gov/programs-surveys/acs/technical-documentation/user-notes/2021-02.html>. Information on 
conﬁdentiality protection, sampling error, nonsampling error, and deﬁnitions is available at <www.census.gov/acs>.
Source: U.S. Census Bureau, 2005–2022 American Community Survey, 1-year estimates.
Recession
/zero.tab
/five.tab/five.tab
/six.tab/zero.tab
/six.tab/five.tab
/seven.tab/zero.tab


### Create a retriever interface using vector store, we'll use it later to construct Q & A chain using LangChain.

In [14]:
# Use similarity searching algorithm and return 3 most relevant documents.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

**Now we have our vector store and retrieval system ready. We then need a large language model (LLM) to process information and answer the question.**

## Open-source LLMs from Hugging Face
**There two ways to utilize Hugging Face LLMs: online and local.**

### Hugging Face Hub
The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together. 

- To use, we should have the huggingface_hub python package installed.
- Set an environment variable called HUGGINGFACEHUB_API_TOKEN with your Hugging Face access token in it.
- Currently, HuggingFace LangChain integration doesn't support the question-answering task, so we can't select HuggingFace QA models for this project. Instead, we select LLMs from the text-generation task category.  

In [15]:
# from langchain_community.llms import HuggingFaceHub

# hf = HuggingFaceHub(
#     repo_id="EleutherAI/gpt-neo-2.7B",
#     model_kwargs={"temperature":0.1, "max_length":500})

#query = """What were the trends in median household income across different states in the United States between 2021 and 2022."""  # Sample question, change to other questions you are interested in.
# hf.invoke(query)

Hugging Face Hub will be slow when you run large models. You can get around this by downloading the model and run it on your local machine. This is the way we use LLM in our project.

### Hugging Face Local Pipelines

Hugging Face models can be run locally through the HuggingFacePipeline class.

- We need to install transformers python package.
- The Mistral-7B-v0.1 Large Language Model (LLM) is a pretrained generative text model with 7 billion parameters. Mistral-7B-v0.1 outperforms Llama-2-13B on all benchmarks tested. Read the [paper](https://arxiv.org/abs/2310.06825).
- Mistral-7B-v0.1's model size is 3.5GB, while Llama-2–13B has 13 billion parameters and 25GB model size.
- In order to use Llama2, you need to request access from Meta. Mistral-7B-v0.1 is publicly available already.

In [ ]:
query = """What were the trends in median household income across different states in the United States between 2021 and 2022."""  # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query)
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 4 documents retrieved which are relevant to the query. Display the first one:

Comparisons
The U.S. median household income 
in 2022 was $74,755, according 
Figure 1.
Median Household Income in the Past 12 Months in the United States: 2005–2022
 
Note: Estimates for 2020 experimental data not shown. For more information on the 2020 experimental data products, 
refer to <www.census.gov/programs-surveys/acs/technical-documentation/user-notes/2021-02.html>. Information on 
conﬁdentiality protection, sampling error, nonsampling error, and deﬁnitions is available at <www.census.gov/acs>.
Source: U.S. Census Bureau, 2005–2022 American Community Survey, 1-year estimates.
Recession
/zero.tab
/five.tab/five.tab
/six.tab/zero.tab
/six.tab/five.tab
/seven.tab/zero.tab


In [18]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--mistralai--Mistral-7B-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [ ]:
llm.invoke(query)

**At a glance, our LLM generates some output that might seem plausible but not accurate or factual. That is because it has not been trained on the census data of recent years.**

- OpenAI GPT-3.5 model (for test purpose only)

In [14]:
# from langchain_openai import ChatOpenAI
# chat = ChatOpenAI(temperature=0)
# chat.invoke(query)
# llm = chat

## Q & A chain 
Now we have both the retrieval system for relevant documents and LLM as QA chatbot ready.

We will take our initial query, together with the relevant documents retrieved based on the results of our similarity search, to create a prompt to feed into the LLM. The LLM will take the initial query as the question and relevant documents as the context information to generate a result.

Luckily, **LangChain** provides an abstraction of the whole pipeline - **RetrievalQA**

**Let's first construct a proper prompt for our task.**

Prompt engineering is another crucial factor in LLM's performance.

In [15]:
prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

Call LangChain's RetrievalQA with the prompt above. 

In [16]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

## Use RetrievalQA invoke method to execute the chain
Note that Input of [invoke method](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html#langchain.chains.retrieval_qa.base.RetrievalQA.invoke) needs to be a dictionary.

In [ ]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})

In [ ]:
print(result['result'])

In [ ]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print("-"*100)

## Conclusion

- Enhanced Accuracy and Relevance: By leveraging external sources, RAG models can generate content that is more accurate, detailed, and relevant to the given context.
- Factuality: It helps in improving the factuality of the generated text, as the information is directly sourced from reliable external databases or knowledge bases.
- Versatility: RAG can be applied to a wide range of NLP tasks, including question answering, text summarization, content creation, and more, enhancing their performance by providing access to a broader range of information.